In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## Mixup

Идея микслоадера заключается в миксовании изображений из разных классов в определенном процентном сотношщении. Тогда выходом модели было бы не приближение определенных классов к 1 или 0, а балансировка между классами. Это позволяет улучшить обобщающую способность. 

Для реализации метода нужна спец.функция потерь смешанного изображения:

$$p * loss(image1) + (1-p) * loss(image2)$$

, где p - это доля первого изображения на смешанном. p лучше всего выбрать из бета-распределения - при таком распределении большинство смешанных изображений в сете будет либо тем либо другим.

В коде у нас появляется два загрузчика данных, что потенциально влечет ошибку, т.к. пакеты не сбалансированы. Это решается случайным перемещением входящего пакета. Это все равно влечет за собой корллизию, так как можно получить смешанное изображение1 и изображение2 с бетапараметром 0,3, а следом в той же партии смешать эти же изображения с параметром 0,7. Чтобы этого не случалось можно реализовать смесь так, что неперемешанная партия всегда будет иметь наивысший компонент (в fast.ai это реализовано через mix_parameters = torch.max(mix_parameters, 1 - mix_parameters)).


In [3]:
# добавим микслоадер
def train(model, optimizer, loss_fn, train_loader, val_loader, mix_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in zip(train_loader, mix_loader):
            ((inputs, targets), (inputs_mix, targets_mix)) = batch
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            inputs_mix = inputs_mix.to(device)
            targets_mix = targets_mix.to(device)
            # вариант со случайным перемещением пакета
            # shuffle = torch.randperm(inputs.size(0))
            # inputs_mix = inputs[shuffle]
            # targets_mix = targets[shuffle]
            
            distribution = torch.distribution.beta.Beta(0.5, 0.5)
            beta = distribution.expand(torch.zeros(batch_size).shape).sample().to(device)
            
            # преобразуем бету в размерност ьвходного тензора [batch_size, channels, height, width]
            mixup = beta[:, None, None, None]
            
            inputs_mixed = mixup * inputs + (1 - mixup) * inputs_mix            
            targets_mixed = beta * targets + (1 - beta) * inputs_mix            
            output_mixed = model(inputs_mixed)
            
            # среднее двух смешанных потерь
            loss = (loss_fn(output, targets) * beta + loss_fn(output, targets_mixed) * (1 - beta)).mean()
            
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))
        
def find_lr(model, loss_fn, optimizer, train_loader, init_value=1e-8, final_value=10.0, device="cpu"):
    number_in_epoch = len(train_loader) - 1
    update_step = (final_value / init_value) ** (1 / number_in_epoch)
    lr = init_value
    optimizer.param_groups[0]["lr"] = lr
    best_loss = 0.0
    batch_num = 0
    losses = []
    log_lrs = []
    for data in train_loader:
        batch_num += 1
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        # Crash out if loss explodes

        if batch_num > 1 and loss > 4 * best_loss:
            if(len(log_lrs) > 20):
                return log_lrs[10:-5], losses[10:-5]
            else:
                return log_lrs, losses

        # Record the best loss

        if loss < best_loss or batch_num == 1:
            best_loss = loss

        # Store the values
        losses.append(loss.item())
        log_lrs.append((lr))

        # Do the backward pass and optimize

        loss.backward()
        optimizer.step()

        # Update the lr for the next step and store

        lr *= update_step
        optimizer.param_groups[0]["lr"] = lr
    if(len(log_lrs) > 20):
        return log_lrs[10:-5], losses[10:-5]
    else:
        return log_lrs, losses  

## Сглаживание меток

При прогнозе мы задаем определенный зазор e (эписилон), который вычитаем из значения вероятности класса. Для этиого нам надо только обернуть функцию потерь.

In [6]:
class LabelSmoothingCrossEntropyLoss(nn.Module):
    
    def __init__(self, epsilon=0.1):
        super(LabelSmoothingCrossEntropyLoss, self).__init__()
        self.epsilon = epsilon
        
    def forward(self, output, target):
        num_classes = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = (-log_preds.sum(dim=-1)).mean()
        nll = F.nll_loss(log_preds, target)
        final_loss = self.epsilon * loss / num_classes + (1 - self.epsilon) * nll
        
        return final_loss